In [14]:
""" Early experiment to parse an obj file and find out what triangles are in it. Then, convert those triangles to STL format

1. OBJ files seem to contain more than triangles. For now, I am ignoring all faces that have more than three points
2. I am only looking at "v" and "f" lines, but that may need to change
3. Not outputing anything yet

STL format is pretty straightforward. It comes in two "flavors" (ASCII and binary), but both have the same data. 
https://en.wikipedia.org/wiki/STL_(file_format)

Binary seems the most common. NOTE: Little Endian!
File starts with some metadata:

UINT8[80] – Header (is generally ignored, apparently)
UINT32 – Number of triangles

Then the triangles; each is representd as 18 bytes:

foreach triangle
REAL32[3] – Normal vector
REAL32[3] – Vertex 1
REAL32[3] – Vertex 2
REAL32[3] – Vertex 3
UINT16 – Attribute byte count
end

3rd component in each triplet on f-field is the index of the normal vector. Note: we assume that the obj file contains 
data in order: v, vn, f

"""
import re
import struct

vertices=[]
normalvectors=[]
triangles=[]
with open('../data/letterMG.obj', 'r') as f:
        for line in f.readlines():
            if line.startswith("v "): # vertices
                data=re.split("\s+", line.strip())
                vertices.append(
                    (float(data[1]), float(data[2]), float(data[3]))
                )
            elif line.startswith("vn "): # normal vectors
                data=re.split("\s+", line.strip())
                normalvectors.append(
                    (float(data[1]), float(data[2]), float(data[3]))
                )
            elif line.startswith("f "): # faces
                data=re.split("\s+", line.strip())
                if len(data) > 4:
                    pass # ignore anything that isn't a triangle right now
                else:
                    triangle=(normalvectors[int(data[1].split("/")[2])],
                              vertices[int(data[1].split("/")[0])], 
                              vertices[int(data[2].split("/")[0])], 
                              vertices[int(data[3].split("/")[0])])
                    triangles.append(triangle)
            else:
                pass # ignore the rest
            
len(triangles)

211

In [15]:
from struct import pack

out=bytearray()
out.extend(b'\x00' * 80)
out.extend(pack("<I", len(triangles))) # number of triangles as unsigned int (32 bits)
for triangle in triangles:
    for c in range(4): # four tuples per triangle
        for p in range(3): # three point per tuple
            out.extend(pack("<f", triangle[c][p])) # each point is a real (32 bits)
    out.extend(pack("<H", 0)) # attributes is an unsigned int (16 bits)  

assert len(out) == 80 + 4 + len(triangles)*50 # sanity check
        
with open('../data/letterMG.stl', 'wb') as f:
    f.write(out)
    
len(out)

10634